In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install x11-utils > /dev/null 2>&1
!pip install gym-super-mario-bros > /dev/null 2>&1
!pip install import-ipynb > /dev/null 2>&1
!pip install -U -q PyDrive > /dev/null 2>&1

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only

import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()
import import_ipynb
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
Drive = GoogleDrive(gauth)
#importando wrapper
# https://drive.google.com/open?id=1hk5V6dPPgcAFNCqTlDwQRM_dlqdxCEwA
my_wrapper = Drive.CreateFile({'id':'1hk5V6dPPgcAFNCqTlDwQRM_dlqdxCEwA'})
my_wrapper.GetContentFile('env_wrapper.ipynb')
import env_wrapper as wrappers
#importando modelo de rede dqn
#https://drive.google.com/open?id=14wjjrd_iI1y1x-N01PJaDBzqC43c1ipd
dqn_model = Drive.CreateFile({'id':'14wjjrd_iI1y1x-N01PJaDBzqC43c1ipd'})
dqn_model.GetContentFile('dqn.ipynb')
import dqn as dqn_model

In [0]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
import gym
import time
import argparse
import numpy as np

import torch
import collections

DEFAULT_ENV_NAME = "SuperMarioBros-1-1-v0"
FPS = 25


if __name__ == "__main__":
    env = wrappers.make_env(DEFAULT_ENV_NAME)
    env = wrap_env(env)

    net = dqn_model.DQN(env.observation_space.shape, env.action_space.n)
    net.load_state_dict(torch.load('best.dat', map_location=lambda storage, loc: storage))

    state = env.reset()
    total_reward = 0.0
    c = collections.Counter()

    while True:
        start_ts = time.time()
        env.render()
        state_v = torch.tensor(np.array([state], copy=False))
        q_vals = net(state_v).data.numpy()[0]
        action = np.argmax(q_vals)
        c[action] += 1
        state, reward, done, _ = env.step(action)
        total_reward += reward
        if done:
            break
        delta = 1/FPS - (time.time() - start_ts)
        if delta > 0:
            time.sleep(delta)

    print("Total reward: %.2f" % total_reward)
    print("Action counts:", c)
    env.close()
    show_video()

/usr/local/lib/python3.6/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Total reward: 1723.00
Action counts: Counter({1: 1765, 3: 82, 4: 56, 0: 44, 2: 21, 5: 19, 6: 18})
